In [1]:
import numpy as np
from scipy.spatial.distance import euclidean

from fastdtw import fastdtw

In [4]:
from scikits.talkbox.features import mfcc
from scipy.io import wavfile

In [5]:
import os
import glob

In [17]:
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
#from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [6]:
folders = ['zero','one','two','three','four','five','six','seven','eight','nine']

In [11]:
def generate_data(folders):
    X, y = [], []
    for label, folder in enumerate(folders):
        path = 'recordings/Data/' + folder + '/*' # need to be changed
        for fn in glob.glob(path):
            print fn                   #just to check which file got problem
            signal = wavfile.read(fn)[1]
            ceps = mfcc(signal)[0]
            mask = np.any(np.isnan(ceps), axis=1)
            ceps = ceps[~mask]                   # remove rows has NaN value
            X.append(ceps)
            y.append(label)
    return np.array(X), np.array(y)

In [12]:
X, y = generate_data(folders)

recordings/Data/zero/100_16-07-17-10-26-47.wav
recordings/Data/zero/101_16-07-17-10-28-12.wav
recordings/Data/zero/102_16-07-17-10-29-29.wav
recordings/Data/zero/103_16-07-17-10-30-44.wav
recordings/Data/zero/104_16-07-17-10-31-59.wav
recordings/Data/zero/105_16-07-17-10-33-10.wav
recordings/Data/zero/107_16-07-17-10-36-31.wav
recordings/Data/zero/108_16-07-17-10-38-27.wav
recordings/Data/zero/10_16-07-16-09-34-37.wav
recordings/Data/zero/11_16-07-16-09-38-50.wav
recordings/Data/zero/12_16-07-16-09-44-18.wav
recordings/Data/zero/14_16-07-16-09-54-18.wav
recordings/Data/zero/15_16-07-16-09-59-04.wav
recordings/Data/zero/16_16-07-16-10-06-19.wav
recordings/Data/zero/17_16-07-16-10-17-51.wav
recordings/Data/zero/18_16-07-16-11-08-41.wav
recordings/Data/zero/19_16-07-16-11-26-54.wav
recordings/Data/zero/20_16-07-16-11-40-28.wav
recordings/Data/zero/21_16-07-16-12-54-26.wav
recordings/Data/zero/22_16-07-16-12-58-20.wav
recordings/Data/zero/23_16-07-17-07-29-42.wav
recordings/Data/zero/24_16

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [46]:
def get_distance(data1, data2):
    distance, path = fastdtw(data1, data2, dist=euclidean)
    return distance

In [53]:
from operator import itemgetter
 
def get_neighbours(training_set, training_label, test_instance, k):
    distances = [_get_tuple_distance(training_instance, label, test_instance) for 
                 (training_instance, label) in zip(training_set, training_label)]
 
    # index 1 is the calculated distance between training_instance and test_instance
    sorted_distances = sorted(distances, key=itemgetter(1))
 
    # extract only training instances
    sorted_training_instances = [tuple[0] for tuple in sorted_distances]
 
    # select first k elements
    return sorted_training_instances[:k]
 
def _get_tuple_distance(training_instance, label, test_instance):
    return (label, get_distance(test_instance, training_instance))

In [74]:
from collections import Counter
 
# 3) given an array of nearest neighbours for a test case, tally up their classes to vote on test case class
 
def get_majority_vote(classes):
    # index 1 is the class
    count = Counter(classes)
    return count.most_common()[0][0]

In [77]:
def dtw_predict(X_test):
    y_pred = []
    for test_instance in X_test:
        classes = get_neighbours(X_train, y_train, test_instance, 5)
        pred = get_majority_vote(classes)
        y_pred.append(pred)
        print(len(y_pred))
    return y_pred

In [78]:
dtw_predict(X_test)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212


[2,
 6,
 8,
 8,
 0,
 8,
 8,
 1,
 3,
 8,
 0,
 0,
 1,
 3,
 1,
 2,
 8,
 8,
 1,
 3,
 9,
 8,
 6,
 0,
 1,
 5,
 8,
 8,
 9,
 6,
 2,
 5,
 8,
 8,
 1,
 2,
 9,
 9,
 6,
 8,
 9,
 9,
 2,
 8,
 0,
 1,
 1,
 0,
 2,
 2,
 8,
 1,
 0,
 1,
 1,
 8,
 2,
 8,
 1,
 4,
 2,
 1,
 1,
 2,
 9,
 3,
 1,
 8,
 8,
 7,
 8,
 8,
 0,
 7,
 5,
 0,
 1,
 9,
 1,
 3,
 2,
 2,
 1,
 1,
 4,
 9,
 0,
 6,
 7,
 1,
 8,
 4,
 2,
 2,
 9,
 5,
 4,
 6,
 5,
 6,
 5,
 1,
 7,
 9,
 0,
 5,
 2,
 2,
 9,
 2,
 1,
 1,
 8,
 4,
 1,
 2,
 6,
 6,
 8,
 1,
 2,
 7,
 8,
 4,
 1,
 4,
 0,
 3,
 3,
 2,
 6,
 7,
 9,
 1,
 5,
 0,
 8,
 0,
 7,
 2,
 0,
 0,
 2,
 1,
 1,
 1,
 9,
 1,
 8,
 7,
 6,
 6,
 1,
 3,
 9,
 0,
 2,
 0,
 2,
 0,
 2,
 2,
 1,
 1,
 8,
 5,
 4,
 6,
 1,
 8,
 2,
 9,
 6,
 1,
 2,
 4,
 0,
 1,
 1,
 9,
 8,
 2,
 1,
 0,
 2,
 2,
 3,
 8,
 1,
 8,
 7,
 2,
 8,
 0,
 6,
 4,
 8,
 2,
 2,
 8,
 6,
 1,
 1,
 0,
 1,
 5,
 0,
 9,
 5,
 0,
 2,
 3]

In [81]:
y_pred = [2,
 6,
 8,
 8,
 0,
 8,
 8,
 1,
 3,
 8,
 0,
 0,
 1,
 3,
 1,
 2,
 8,
 8,
 1,
 3,
 9,
 8,
 6,
 0,
 1,
 5,
 8,
 8,
 9,
 6,
 2,
 5,
 8,
 8,
 1,
 2,
 9,
 9,
 6,
 8,
 9,
 9,
 2,
 8,
 0,
 1,
 1,
 0,
 2,
 2,
 8,
 1,
 0,
 1,
 1,
 8,
 2,
 8,
 1,
 4,
 2,
 1,
 1,
 2,
 9,
 3,
 1,
 8,
 8,
 7,
 8,
 8,
 0,
 7,
 5,
 0,
 1,
 9,
 1,
 3,
 2,
 2,
 1,
 1,
 4,
 9,
 0,
 6,
 7,
 1,
 8,
 4,
 2,
 2,
 9,
 5,
 4,
 6,
 5,
 6,
 5,
 1,
 7,
 9,
 0,
 5,
 2,
 2,
 9,
 2,
 1,
 1,
 8,
 4,
 1,
 2,
 6,
 6,
 8,
 1,
 2,
 7,
 8,
 4,
 1,
 4,
 0,
 3,
 3,
 2,
 6,
 7,
 9,
 1,
 5,
 0,
 8,
 0,
 7,
 2,
 0,
 0,
 2,
 1,
 1,
 1,
 9,
 1,
 8,
 7,
 6,
 6,
 1,
 3,
 9,
 0,
 2,
 0,
 2,
 0,
 2,
 2,
 1,
 1,
 8,
 5,
 4,
 6,
 1,
 8,
 2,
 9,
 6,
 1,
 2,
 4,
 0,
 1,
 1,
 9,
 8,
 2,
 1,
 0,
 2,
 2,
 3,
 8,
 1,
 8,
 7,
 2,
 8,
 0,
 6,
 4,
 8,
 2,
 2,
 8,
 6,
 1,
 1,
 0,
 1,
 5,
 0,
 9,
 5,
 0,
 2,
 3]

In [82]:
confusion_matrix(y_test, y_pred)

array([[15,  2,  4,  2,  1,  0,  0,  1,  0,  1],
       [ 0, 12,  2,  0,  0,  0,  0,  0,  3,  1],
       [ 2,  4, 11,  1,  0,  0,  1,  0,  0,  3],
       [ 1,  4, 10,  7,  1,  0,  0,  0,  0,  2],
       [ 1,  7,  2,  0,  2,  0,  3,  1,  6,  0],
       [ 0,  3,  2,  0,  0,  6,  0,  0,  9,  1],
       [ 2,  3,  2,  0,  2,  1,  6,  0,  0,  3],
       [ 2,  3,  0,  0,  3,  0,  4,  6,  1,  0],
       [ 0,  3,  0,  0,  0,  4,  0,  1, 14,  2],
       [ 2,  2,  2,  0,  1,  0,  2,  0,  2,  5]])

In [83]:
accuracy_score(y_test, y_pred)

0.39622641509433965